# テキスト分類

* DistilBERTをFine-tuningしてTwitterデータの感情検出器を作る
* angry, love, fear, joy, sadness, surpriseの6感情

## データセット

* [emotionsデータセット](https://huggingface.co/datasets/emotion)

```
Emotion は、怒り、恐れ、喜び、愛、悲しみ、驚きの 6 つの基本的な感情を含む英語の Twitter メッセージのデータセットです。詳細については、論文を参照してください
```

### Hugging Face Datasets

* `list_datasets()` でデータセット一覧を確認できる
* `load_dataset()` でダウンロードできる

In [ ]:
from datasets import list_datasets

all_datasets = list_datasets()
print(f"There are {len(all_datasets)} datasets currently available on the Hub")
print(f"The first 10 are: {all_datasets[:10]}")

In [ ]:
from datasets import load_dataset

emotions = load_dataset("emotion")

In [ ]:
emotions

In [ ]:
train_ds = emotions["train"]
train_ds

In [ ]:
len(train_ds)

In [ ]:
train_ds[0]

In [ ]:
train_ds.column_names

In [ ]:
train_ds.features

In [ ]:
train_ds[:5]

In [ ]:
train_ds["text"][:5]

### DataFrameへの変換

* `set_format()` でDatasetの出力形式を変更できる
* `int2str()` でラベルIDを文字列に変換できる

In [ ]:
import pandas as pd

emotions.set_format(type="pandas")

# 出力がpandasのDataFrameになった
emotions["train"][0]

In [ ]:
# trainの全データを取得
df = emotions["train"][:]
df.head()

In [ ]:
emotions["train"].features["label"]

In [ ]:
emotions["train"].features["label"].int2str([0, 2])

In [ ]:
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()

### クラス分布の確認

In [ ]:
import matplotlib.pyplot as plt

df["label_name"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

### ツイートの長さはどれくらい？

* Transformerモデルには最大コンテキストサイズという入力系列長の制限がある
* DistilBERTは512トークン

In [ ]:
df["text"].str.split()[0]

In [ ]:
df["Words Per Tweet"] = df["text"].str.split().apply(len)
df.boxplot("Words Per Tweet", by="label_name", grid=False, showfliers=False, color="black")
plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
# データセットの出力形式を戻す
emotions.reset_format()

## テキストからトークンへ

* 通常、最適な単語分割はコーパスから学習する
* もっともシンプルなやり方は文字トークン化と単語トークン化
* この2つのやり方もデータによって変わるので学習と言ってもよさそう

### 文字トークン化

In [ ]:
text = "Tokenizing text is a core task of NLP."
tokenized_text = list(text)
print(tokenized_text)

In [ ]:
print(sorted(set(tokenized_text)))

In [ ]:
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)

In [ ]:
len(token2idx)

In [ ]:
input_ids = [token2idx[token] for token in tokenized_text]
print(input_ids)

In [ ]:
# one-hotベクトルへの変換
import torch
import torch.nn.functional as F

input_ids = torch.tensor(input_ids)
one_hot_encodings = F.one_hot(input_ids, num_classes=len(token2idx))
one_hot_encodings.shape

### サブワードトークン化

* 文字トークン化と単語トークン化の中間
* コーパスからトークン化を学習する
* 頻出単語は単語として使う、そうでないものはより小さな単位に分割する
* WordPiece: BERTとDistilBERTのTokenizer
* `AutoTokenizer.from_pretrained()` を使うと指定したモデルのTokenizerをロードできる
* [Autoがついている場合](https://huggingface.co/docs/transformers/model_doc/auto)はモデル名から自動判定する
* モデルに対応するTokenizerを使う必要がある

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenizer

In [ ]:
# Autoクラスを使わずにモデルごとのクラスも使える
from transformers import DistilBertTokenizer

distilbert_tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)
distilbert_tokenizer

* Tokenizerをメソッドとして使うとinput_idsに変換できる
* `convert_ids_to_tokens` を使うとinput_idsをトークンに戻せる

In [ ]:
text = "Tokenizing text is a core task of NLP."
encoded_text = tokenizer(text)
print(encoded_text)

* [CLS] と [SEP] のような特別なトークンが付与される（モデルによって異なる）
* `##` は分割されたトークン、文字列に変換するときは前のトークンとマージされる
* 一般的でない単語は `##` で分割されやすい

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

* `convert_to_tokens_to_string()` でトークン列を文字列に変換できる

In [ ]:
print(tokenizer.convert_tokens_to_string(tokens))

In [ ]:
# ボキャブラリーサイズ
tokenizer.vocab_size

In [ ]:
# 最大コンテキストサイズ
tokenizer.model_max_length

In [ ]:
# モデルのforwardパスで期待するフィールド名
tokenizer.model_input_names